In [59]:
import vrep 
import sys
import time 
import numpy as np
from tank import *

import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [73]:
def parallel_zero_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):
    # distance1 = EN, distance2 = ES
    distance2 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance2')
    distance1 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance1')

    speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'speed')

    distance1['very_close'] = fuzz.trapmf(distance1.universe, [0, 0, 1.5, 1.55])
    distance1['medium']     = fuzz.trapmf(distance1.universe, [1.5, 1.55, 2.5, 2.51])
    distance1['far']        = fuzz.trapmf(distance1.universe, [2.5, 2.51, 6, 6])

    distance2['very_close'] = fuzz.trapmf(distance2.universe, [0, 0, 1.1, 2])
    distance2['far']        = fuzz.trapmf(distance2.universe, [1.1, 2, 6, 6])

    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])
    speed['slow'] = fuzz.trimf(speed.universe, [0, 3, 5])
    speed['fast'] = fuzz.trimf(speed.universe, [5, 6, 7])

    rules = (
        ctrl.Rule(distance1['far'] & distance2['far'], speed['fast']),
        ctrl.Rule(distance1['very_close'] & distance2['very_close'], speed['fast']),
        ctrl.Rule(distance1['very_close'] & distance2['far'], speed['fast']),
        ctrl.Rule(distance1['far'] & distance2['very_close'], speed['slow']),
        ctrl.Rule(distance1['medium'] & distance2['far'], speed['slow']),
        ctrl.Rule(distance1['medium'] & distance2['very_close'], speed['stop']),
    )

    speed_ctrl = ctrl.ControlSystem(rules)
    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)

    speed_sim.input['distance2'] = distance_ES_input
    speed_sim.input['distance1'] = distance_EN_input

    speed_sim.compute()

    return speed_sim.output['speed']

def parallel_first_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):
    # distance1 = SE, distance2 = ES
    distance2 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance2')
    distance1 = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance1')

    speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'speed')

    distance1['very_close'] = fuzz.trapmf(distance1.universe, [0, 0, 2.4, 3])
    distance1['far']        = fuzz.trapmf(distance1.universe, [2.4, 3, 6, 6])

    distance2['very_close'] = fuzz.trapmf(distance2.universe, [0, 0, 1.1, 2])
    distance2['far']        = fuzz.trapmf(distance2.universe, [1.1, 2, 6, 6])

    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])
    speed['slow'] = fuzz.trimf(speed.universe, [0, 3, 5])
    speed['fast'] = fuzz.trimf(speed.universe, [5, 6, 7])

    rules = (
        ctrl.Rule(distance1['far'] & distance2['far'], speed['fast']),
        ctrl.Rule(distance1['far'] & distance2['very_close'], speed['stop']),
        ctrl.Rule(distance1['very_close'] & distance2['far'], speed['slow']),
        ctrl.Rule(distance1['very_close'] & distance2['very_close'], speed['fast'])
    )

    speed_ctrl = ctrl.ControlSystem(rules)
    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)

    speed_sim.input['distance2'] = distance_ES_input
    speed_sim.input['distance1'] = distance_SE_input

    speed_sim.compute()

    return speed_sim.output['speed']

def parallel_second_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):

    distance_input = min(distance_ES_input, distance_SE_input)

    distance = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance')
    speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'speed')

    distance['very_close'] = fuzz.trapmf(distance.universe, [0, 0, 0.79, 0.80])
    distance['far']        = fuzz.trapmf(distance.universe, [0.79, 0.80, 6, 6])

    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])
    speed['slow'] = fuzz.trimf(speed.universe, [0, 1, 2])
    speed['fast'] = fuzz.trimf(speed.universe, [5, 6, 7])

    rules = (
        ctrl.Rule(distance['far'], speed['fast']),
        ctrl.Rule(distance['very_close'], speed['stop']),
    )

    speed_ctrl = ctrl.ControlSystem(rules)
    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)
    speed_sim.input['distance'] = distance_input
    speed_sim.compute()

    return speed_sim.output['speed']

def parallel_third_stage(distance_SW_input, distance_WS_input, tank):

    distance_input = min(distance_SW_input, distance_WS_input)
    distance = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance')

    speed = ctrl.Consequent(np.arange(-1, 7, 0.01), 'speed')

    distance['very_close'] = fuzz.trapmf(distance.universe, [0, 0, 1.1, 2])
    distance['far']        = fuzz.trapmf(distance.universe, [1.1, 2, 6, 6])

    speed['stop'] = fuzz.trimf(speed.universe, [-1, 0, 1])
    speed['slow'] = fuzz.trimf(speed.universe, [0, 1, 2])
    speed['fast'] = fuzz.trimf(speed.universe, [5, 6, 7])

    rules = (
        ctrl.Rule(distance['far'], speed['fast']),
        ctrl.Rule(distance['very_close'], speed['stop']),
    )

    speed_ctrl = ctrl.ControlSystem(rules)
    speed_sim = ctrl.ControlSystemSimulation(speed_ctrl)
    speed_sim.input['distance'] = distance_input
    speed_sim.compute()

    return speed_sim.output['speed']

In [74]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [75]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#first read with streaming
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [ ]:
distances = dict()
detection_states = dict()

for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState
current_stage = 0
t = time.time()
while (time.time()-t)<100:

    for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState

    if current_stage == 0:
        fuzzy_speed = parallel_zero_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if fuzzy_speed < 3:
            tank.forward(0)
            current_stage += 1
        else:
            tank.forward(fuzzy_speed)

    if current_stage == 1:
        fuzzy_speed = parallel_first_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if fuzzy_speed < 1:
            tank.forward(0)
            current_stage += 1
        else:
            tank.forward(fuzzy_speed)

    if current_stage == 2:
        fuzzy_speed = parallel_second_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if fuzzy_speed < 1:
            tank.forward(0)
            current_stage += 1
        else: # going back, right wheel is slowed
            tank.leftvelocity = -fuzzy_speed
            tank.rightvelocity = -fuzzy_speed / 6
            tank.setVelocity()

    if current_stage == 3:
        fuzzy_speed = parallel_third_stage(distances['SW'], distances['WS'], tank)
        if fuzzy_speed < 1:
            tank.forward(0)
            break
        else: # now left wheel is slowed
            tank.leftvelocity = -fuzzy_speed / (8/3)
            tank.rightvelocity = -fuzzy_speed
            tank.setVelocity()